# **Đồ án: Phân loại và đự đoán khả năng đơn hàng bị từ chối trong Thương mại điện tử**


<a id="background"></a>
# **1. Đặt vấn đề:**

- Bối cảnh nghiên cứu: Trong lĩnh vực thương mại điện tử (E-commerce), tỷ lệ đơn hàng bị từ chối hoặc hủy bỏ (Order Rejection) là một thách thức lớn ảnh hưởng trực tiếp đến doanh thu và chi phí vận hành. Việc phân tích hành vi khách hàng liên quan đến các quyết định này là cơ sở quan trọng để tối ưu hóa quy trình hậu cần.

- Câu hỏi nghiên cứu: Khả năng một đơn hàng sau khi đặt mua sẽ bị từ chối/hủy là bao nhiêu? Và yếu tố nào gây ra những sự từ chối/hủy này?


<a id="objectives"></a>
# **2.Mục tiêu nghiên cứu:**

- Ứng dụng và so sánh các thuật toán Học máy (Machine Learning) để xây dựng mô hình phân lớp có khả năng dự đoán chính xác trạng thái rủi ro (Bị từ chối/Hủy) của đơn hàng mới.

- Xác định và xếp hạng mức độ quan trọng của các đặc trưng ảnh hưởng đến kết quả dự báo


<a id="data_extraction"></a>
# **3. TRÍCH XUẤT DỮ LIỆU**
<a id="import_libraries"></a>
## **3.a. Import (Nạp) các thư viện liên quan**



In [ ]:
!pip install -q scikit-learn==1.3.2

# Cài 'imbalanced-learn' bản 0.11.0 
!pip install -q imbalanced-learn==0.11.0

# Cài 'pandas' bản 2.0.3 
!pip install -q pandas==2.0.3

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from glob import glob
import warnings

# Thư viện cho Tiền xử lý và Cân bằng dữ liệu
from imblearn.over_sampling import SMOTENC # Sử dụng SMOTE cho dữ liệu hỗn hợp
from sklearn.preprocessing import OneHotEncoder, StandardScaler # Đổi sang StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

# Thư viện cho Mô hình hóa và Đánh giá
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, roc_auc_score, classification_report, accuracy_score
from sklearn import set_config

# Cấu hình hiển thị
sns.set()
set_config(display="diagram")
warnings.simplefilter(action="ignore", category="FutureWarning") # Ẩn các cảnh báo phiên bản

<a id="check_file_names"></a>
## **3.b. Kiểm tra tên file trong đường dẫn dataset**



In [ ]:
# Sử dụng glob để tìm TẤT CẢ file . csv trong thư mục input
# Pattern: "/kaggle/input/unlock-profits-with-e-commerce-sales-data/*.csv"
# Tìm mọi file có đuôi .csv trong thư mục này
filepath_list = glob("/kaggle/input/unlock-profits-with-e-commerce-sales-data/*.csv")

# In ra danh sách các file tìm được (để kiểm tra)
print(f"Tên của tất cả {len(filepath_list)} file csv trong thư mục là: ")
for number, filepath_name in enumerate(filepath_list, start=1):
    # Cắt bỏ phần đường dẫn dài, chỉ lấy tên file
    # rsplit(sep='/', maxsplit=1): Cắt từ phải sang trái, chỉ cắt 1 lần
    print(f" {number}.{filepath_name.rsplit(sep='/', maxsplit=1)[1]}")

<a id="read_dataset_to_dataframe"></a>
## **3.c. Đọc dataset vào một dataframe pandas tạm thời**


In [ ]:
# Đọc file CSV đầu tiên (filepath_list[0]) vào DataFrame
# - low_memory=False: Tắt cảnh báo khi đọc file lớn (tự động detect dtype)
dataframe = pd.read_csv(filepath_list[0], low_memory=False)

# Hiển thị thông tin tổng quan của DataFrame
# → Số dòng, số cột, kiểu dữ liệu, số giá trị non-null
dataframe.info()

<a id="data_transformation_wrangle"></a>
# 4. DATA TRANSFORMATION**
<a id="first_draft_wrangle"></a>
## 4.a. First Draft Wrangle Function
- Hàm này dùng để làm sạch sơ bộ (preliminary cleansing) dataframe và chưa phải là hàm làm sạch cuối cùng, sẽ cải tiển sau EDA

- Các bước:

    1. Chuẩn hóa tên cột (lowercase, xóa space, xóa dash)
    2. Xóa các cột không cần thiết
    3.  Điền giá trị NaN cho location data
    4. Chuẩn hóa tên bang (shipstate)
    5. Xóa dòng trùng lặp

In [ ]:
def first_draft_wrangle(dataframe):
    
 # BƯỚC 1: Tạo bản sao (copy) để tránh thay đổi dataframe gốc
    df = dataframe.copy()
    
 # BƯỚC 2: Chuẩn hóa tên cột
    # Ví dụ: "Order ID" → "orderid", "ship-service-level" → "shipservicelevel"
    col = [element.lower().replace(" ", "").replace("-", "") 
           for element in df.columns]
    df.columns = col
    
# BƯỚC 3: Xóa các cột "rác" (không cần cho phân tích)
    # - index: Cột index dư thừa
    # - date: Không phân tích time series
    # - fulfilledby: Dư thừa (có cột 'fulfilment' rồi)
    # - currency: Tất cả đều INR (Indian Rupees)
    # - unnamed:22: Cột rỗng (data entry error)
    # - promotionids: Quá nhiều giá trị unique (high cardinality)
    # - courierstatus: Không liên quan trực tiếp
    # - shipcountry: Tất cả đều India
    df.drop(["index", "date", "fulfilledby", "currency", "unnamed:22",
             "promotionids", "courierstatus", "shipcountry"],
            axis="columns", inplace=True)
    
# BƯỚC 4: Xử lý giá trị thiếu (NaN) cho location data
    # Điền "unknown" cho bang và thành phố bị thiếu
    df["shipstate"].fillna("unknown", inplace=True)
    df["shipcity"].fillna("unknown", inplace=True)
    
    # Điền 0 cho postal code thiếu, sau đó chuyển sang object (categorical)
    df["shippostalcode"].fillna(0, inplace=True)
    df["shippostalcode"] = df["shippostalcode"].astype(int).astype(object)
    
# BƯỚC 5: Chuẩn hóa tên bang (shipstate)
    # 5.1. Viết hoa tất cả (để dễ so sánh)
    df["shipstate"] = df["shipstate"].str.upper()
    
    # 5.2.Gom các tên bị gõ sai/viết tắt về tên chuẩn
    # Ví dụ: "RJ" → "RAJASTHAN", "DELHI" → "NEW DELHI"
    df["shipstate"].replace({
        "PONDICHERRY": "PUDUCHERRY",
        "RAJSHTHAN": "RAJASTHAN",    # Typo
        "RAJSTHAN": "RAJASTHAN",      # Typo
        "RJ": "RAJASTHAN",            # Abbreviation
        "PB": "PUNJAB",               # Abbreviation
        "PUNJAB/MOHALI/ZIRAKPUR": "PUNJAB",
        "ORISSA": "ODISHA",           # Old name
        "DELHI": "NEW DELHI",
        "NL": "UNKNOWN",              # Invalid code
        "APO": "UNKNOWN",             # Military address
        "AR": "UNKNOWN"               # Invalid
    }, inplace=True)
    
# BƯỚC 6: Xóa các dòng trùng lặp (nếu có)
    df.drop_duplicates(inplace=True)
    
# BƯỚC 7: Trả về DataFrame đã làm sạch
    return df

<a id="eda"></a>
## **4.b. Phân tích Khám phá Dữ liệu (EDA)**

<a id="preliminary_data_cleaning"></a>
### **4.b.i. Làm sạch dữ liệu sơ bộ**<br>
Sử dụng hàm first_draft_wrangle(), chúng ta đọc và lưu trữ dataset đã được làm sạch sơ bộ vào dataframe tên là "df".

In [ ]:
df = first_draft_wrangle(dataframe)
#Kiểm tra kết quả sau khi làm sạch 
df.info()

<a id="orderid"></a>
### <br>**4.b.ii. Phân tích cột "orderid" (Mã đơn hàng).**<br>
**Mục đích:**
- Kiểm tra xem có bao nhiêu đơn hàng "mua 1 sản phẩm" vs "mua nhiều sản phẩm"
- Quyết định có nên giữ lại đơn hàng "nhiều sản phẩm" hay không

In [ ]:

# TẠO DATAFRAME CHỨA CÁC DÒNG CÓ ORDERID LẶP LẠI
# - duplicated(keep=False): Đánh dấu TẤT CẢ các dòng có orderid lặp
#   (keep=False → không giữ dòng nào, đánh dấu hết)
# Ví dụ: orderid "ABC123" xuất hiện 3 lần → 3 dòng đều được đánh dấu True
orderid_repeat_rows = df[df["orderid"].duplicated(keep=False)]

# ĐẾM SỐ LƯỢNG ORDERID (DUY NHẤT) BỊ LẶP
# nunique() = number of unique values
unique_orderid_repeat_count = orderid_repeat_rows["orderid"].nunique()

# ĐẾM TỔNG SỐ DÒNG (ROWS) BỊ CHIẾM BỞI CÁC ĐƠN HÀNG LẶP
orderid_repeat_rows_count = len(orderid_repeat_rows)

# ĐẾM SỐ DÒNG CỦA CÁC ĐƠN HÀNG "KHÔNG LẶP" (Mua 1 món)
# duplicated(keep=False) == False: Chỉ lấy các dòng orderid KHÔNG lặp
orderid_nonrepeat_rows = df[df["orderid"].duplicated(keep=False) == False]
orderid_nonrepeat_rows_count = len(orderid_nonrepeat_rows)

# IN KẾT QUẢ
print(f"1.  Số lượng orderid (duy nhất) mua nhiều sản phẩm: {unique_orderid_repeat_count}\n"
      f"2. Tổng số dòng của các orderid mua nhiều sản phẩm: {orderid_repeat_rows_count}\n"
      f"3. Số lượng orderid (duy nhất) mua một sản phẩm: {orderid_nonrepeat_rows_count}")

Sau đó tiến hành vẽ biểu đồ cột thể hiện tỷ lệ của hai loại đơn hàng này

In [ ]:
# Tạo figure và axes
fig, ax = plt.subplots(figsize=(7, 5))

# TÍNH TỶ LỆ % của 2 loại đơn hàng
# - value_counts(normalize=True): Đếm + tính tỷ lệ
# - *100: Chuyển thành %
# - round(... , 2): Làm tròn 2 số lẻ
orderid_repeat_rows = df[df["orderid"].duplicated(keep=False)]
unique_orderid_repeat_list = orderid_repeat_rows["orderid"].unique()
plot_dataseries = round(
    df["orderid"].isin(unique_orderid_repeat_list).value_counts(normalize=True) * 100, 
    2
)

# VẼ BIỂU ĐỒ CỘT
plot_dataseries.plot(kind="bar", ax=ax)

# TIÊU ĐỀ VÀ NHÃN
plt.title("Tỷ lệ các Đơn hàng Mua 1 sản phẩm \nvà Mua nhiều sản phẩm", fontsize=16)
plt.ylabel("Tỷ lệ", fontsize=14)
plt.xlabel("Loại đơn hàng", fontsize=14)
plt.xticks(ticks=plot_dataseries.index,
           labels=["Đơn hàng 1 sản phẩm", "Đơn hàng nhiều sản phẩm"], 
           rotation="horizontal")
plt.yticks(ticks=[20, 40, 60, 80, 100, 120], 
           labels=["20%", "40%", "60%", "80%", "100%", "120%"])

# THÊM NHÃN DỮ LIỆU (Data labels)
# Tạo nhãn % (ví dụ: "88. 03%")
data_label = plot_dataseries.astype(str).str.cat(np.full((2,), "%"), sep="")
# Tạo nhãn số đếm (ví dụ: "35000")
count_label = pd.Series(df["orderid"].isin(unique_orderid_repeat_list).value_counts()).astype("str")

# Vẽ nhãn % lên biểu đồ
for x, y in enumerate(plot_dataseries):
    plt.text(x, y-10, data_label[x], color="white",
             fontweight=700, fontsize=14, horizontalalignment="center")

# Vẽ nhãn số đếm lên biểu đồ
for x, y in enumerate(plot_dataseries):
    plt.text(x, y+5, ("Số lượng:\n" + count_label[x]), color="Darkgreen",
             fontweight=700, fontsize=14, horizontalalignment="center")

plt.show()

#### NHẬN XÉT:

**Quan sát:**
- 88.04% (gần 90%) dữ liệu là từ các đơn hàng "mua 1 sản phẩm" (single-product order)
- Chỉ có 11.96% là đơn hàng "mua nhiều sản phẩm" (multi-product order)

**Lý do:**
- "Multi-product orders introduce complexity: 
  - Có thể 1 sản phẩm bị reject, 1 sản phẩm OK
  - Khó xác định target rõ ràng (rejected hay không?)
  - Analysis riêng biệt cho multi-product orders recommended"

**Quyết định:**
**Xóa các đơn hàng "mua nhiều sản phẩm"**
- Lý do 1: Chiếm tỷ lệ nhỏ (chỉ 12%)
- Lý do 2: Khó xác định target (rejected hay không unclear)
- Lý do 3: Focus vào đa số (88%) để có kết quả rõ ràng hơn

**Sau đó xóa cột "orderid"** Vì sau khi chỉ giữ "đơn 1 món" → orderid không còn cần thiết (mỗi dòng = 1 đơn riêng biệt)

In [ ]:
# Loại bỏ các đơn hàng có chứa nhiều sản phẩm (chỉ giữ lại đơn hàng mua 1 sản phẩm duy nhất)
df = df[df["orderid"].duplicated(keep = False) == False]
# drop redundant column "orderid"
# Xóa cột "orderid" vì nó đã trở nên dư thừa 
df.drop("orderid", axis = 1, inplace = True)
# Xem lại thông tin dataframe sau khi xử lý
df.info()

### **4.b.iii. Tạo cột "rejected" (Target Variable - Biến mục tiêu)**

**Mục đích:**
- Tạo biến mục tiêu nhị phân (binary target) cho bài toán classification
- Chuyển từ cột "status" (có nhiều giá trị) → cột "rejected" (chỉ có 0/1)
- Target Definition:
  - **1 (Rejected):** Các đơn hàng bị hủy/trả lại
  - **0 (Not Rejected):** Các đơn hàng giao thành công

**Phương pháp: Binary Encoding**
- **Bước 1:** Xác định các giá trị "status" rõ ràng (confirmed outcomes)
- **Bước 2:** Loại bỏ các status "lơ lửng" (Pending, Shipped without delivery confirmation)
- **Bước 3:** Encode: Cancelled', 'Shipped - Returned to Seller', 'Shipped - Rejected by Buyer', 'Shipped - Returning to Seller' → 1, 'Shipped - Delivered to Buyer' → 0

**Lý do loại bỏ status "lơ lửng":**
- "Pending": Chưa biết kết quả cuối cùng
- "Shipped": Có thể delivered, có thể returned → không chắc chắn
- → Giữ lại sẽ gây **label noise** (nhiễu nhãn) → giảm accuracy của model

In [ ]:
# Lấy tất cả giá trị unique (duy nhất) trong cột "status"
# → Để xác định cách phân loại "Rejected" vs "Not Rejected"
print("Các giá trị unique trong cột 'status':")
print(df["status"].unique())

In [ ]:
# Xóa các dòng có status không rõ ràng
# 1. Định nghĩa các status có kết quả cuối cùng rõ ràng
known_value = ["Cancelled", 'Shipped - Returned to Seller','Shipped - Rejected by Buyer',
            'Shipped - Returning to Seller','Shipped - Delivered to Buyer']
# 2. Lọc dataframe, chỉ giữ lại các dòng có status nằm trong danh sách known_value
df = df[df["status"].isin(known_value)]    

# 3. TẠO CỘT "rejected" (Binary: 1=Rejected, 0=Not Rejected)
rejected = ["Cancelled", 'Shipped - Returned to Seller','Shipped - Rejected by Buyer',
            'Shipped - Returning to Seller']
# 4. Nếu status nằm trong danh sách "rejected" -> gán giá trị 1, ngược lại là 0
df["rejected"] = df["status"].isin(rejected).astype(int)    # đổi kiểu dữ liệu sang số nguyên (int) 

# 5. Xóa (drop) cột "status" gốc đi (vì đã có cột "rejected" mới)
df.drop("status",axis = "columns", inplace = True)

<br>Vẽ biểu đồ cột biểu diễn tỷ lệ của 2 status này

In [ ]:
# vẽ biểu đồ cột dùng thư viện matplotlib
fig,ax = plt.subplots(figsize = (7,5))

# tính toán và lưu trữ tỷ lệ % của 2 lớp (0 và 1) vào một pandas.Series
plot_dataseries = round(df["rejected"].value_counts(normalize = True)*100,2)

# vẽ biểu đồ cột
plot_dataseries.plot(kind = "bar",ax =ax)
# Đặt Tiêu đề và Nhãn 
plt.title("Tỷ lệ các đơn hàng Bị từ chối và Không bị từ chối", fontsize = 14)
plt.ylabel("Tỷ lệ (%)", fontsize = 12)
plt.xlabel("Trạng thái", fontsize = 12)
plt.xticks(ticks = range(len(plot_dataseries)),
           labels = ["Không bị từ chối", "Bị từ chối"], rotation = "horizontal") 
plt.yticks(ticks = [20,40,60,80,100], labels = ["20%","40%","60%","80%","100%"])

# tạo một series khác để hiển thị nhãn % (ví dụ: "58.2%")
data_label = plot_dataseries.astype(str).str.cat(np.full((2,),"%"), sep = "")

# tạo thêm một series nữa để hiển thị số đếm (ví dụ: 20000)
data_count = df["rejected"].value_counts()

# thêm/vẽ nhãn % lên biểu đồ
for x,y in enumerate(plot_dataseries):
    plt.text(x,y-10,data_label[x],color = "white",
             fontweight = 700,fontsize = 13, horizontalalignment = "center")

# thêm/vẽ nhãn số đếm (Count) lên biểu đồ
for x,y in enumerate(plot_dataseries):
    plt.text(x,y+5,"Số lượng:\n" + str(data_count[x]),fontweight = 700,
             fontsize = 13,horizontalalignment = "center")
    
plt.show()

#### NHẬN XÉT VỀ PHÂN PHỐI TARGET:

**Quan sát:**
- **57.7%** là "Not Rejected" (0)
- **42.3%** là "Rejected" (1)
- Tỷ lệ: **58:42** (không quá mất cân bằng)
  
**Phân tích:**
**Ưu điểm của tỷ lệ hiện tại (58:42):**
- Không quá mất cân bằng (slight imbalance)
- Class thiểu số (42%) vẫn có đủ samples để học
- Model có thể học được patterns của cả 2 classes

**Nhược điểm:**
- Model có thể hơi "thiên vị" (biased) về class 0
- Metrics như Accuracy có thể misleading (model đoán toàn 0 vẫn được 58% accuracy)

**Quyết định:**

**KHÔNG xử lý ngay ở đây vì:**
- Data hiện tại là **TOÀN BỘ dataset** (chưa chia train/test)
- Nếu oversample bây giờ → sẽ có data leakage:
  - Train và Test đều có synthetic samples
  - Model sẽ "gian lận" (thấy test data trong quá trình train)
  - Accuracy giả tạo (inflated accuracy)

**SẼ xử lý ở Bước 6. b (sau khi chia train/test):**
1. Chia dataset → Train/Test
2. CHỈ oversample (SMOTE) trên TRAIN set
3. Giữ nguyên Test set (không động vào)
4. Đảm bảo đánh giá công bằng

### **4.b.iv.  Kiểm tra High Cardinality Features**
**Mục đích:**
- Xác định các cột categorical có **quá nhiều** giá trị unique (high cardinality)
- Quyết định xem có nên giữ lại hay loại bỏ

**Phương pháp: Cardinality Check**
- Đếm số unique values cho mỗi cột categorical
- Identify các cột có cardinality quá cao (> 100)

In [ ]:
# "Cardinality > 50 được coi là cao, > 500 là rất cao"
print("Tiến hành kiểm tra số lượng giá trị duy nhất (Cardinality) của các đặc trưng phân loại...")
print()
#  Đếm số unique values cho MỖI cột có dtype='object' (categorical)
# .select_dtypes(include="object"): Chọn các cột biến định danh/phân loại
# .nunique(): Đếm số giá trị duy nhất cho mỗi cột
cardinality_series = df.select_dtypes(include="object").nunique()

# Sắp xếp từ cao xuống thấp để thuận tiện quan sát
cardinality_series = cardinality_series.sort_values(ascending=False)

print("Thống kê số lượng giá trị duy nhất của từng đặc trưng:")
print(cardinality_series)

# Xác định các cột có Cardinality RẤT CAO (> 100)
high_cardinality_cols = cardinality_series[cardinality_series > 100]

if len(high_cardinality_cols) > 0:
    print("\n CÁC ĐẶC TRƯNG CÓ Cardinality cao (> 100 giá trị duy nhất):")
    for col, count in high_cardinality_cols.items():
        print(f"   • Đặc trưng '{col}': {count} giá trị duy nhất")
else:
    print("\n Không phát hiện đặc trưng nào có Cardinality bất thường (High Cardinality).")

#### NHẬN XÉT & QUYẾT ĐỊNH:

**Quan sát:**

Các cột sau có **High Cardinality** (quá nhiều unique values):`sku`, `style`, `shipcity`, `shippostalcode`, `asin`. -->Xóa

**Giữ lại các features có cardinality hợp lý:**
- `category` (~5 values) 
- `fulfilment` (2 values) 
- `shipservicelevel` (2 values) 
- `b2b` (2 values) 
- `region` (sẽ tạo - 6 values) 

**Kết quả:**
- Giảm complexity
- Tránh overfitting
- Faster training
- Better generalization

In [ ]:
df.drop(["style","sku","shipcity","shippostalcode","asin"],axis = 1, inplace = True)

### **4. b.v. Kiểm tra các cột không phải Categorical (Numerical + Others)**

**Mục đích:**
- Xác định các cột có dtype khác "object" (không phải categorical)
- Kiểm tra: Numerical features (số), Boolean features (True/False), etc.

**Phương pháp: Dtype Inspection**
- Sử dụng `. select_dtypes(exclude="object")` để lọc
- Kiểm tra: dtype, số NaN, range values

In [ ]:
#  hiển thị thông tin các cột KHÔNG phải là 'object' (tức là cột số, cột boolean)
df.select_dtypes(exclude = "object").info()
df.select_dtypes(exclude="object").describe()

<a id="qty"></a>
### <br>**4.b.vi.Phân tích cột "qty" (Số lượng mỗi đơn)**
**Mục đích:**
- Kiểm tra phân phối của số lượng sản phẩm trong mỗi đơn hàng
- Xác định có bất thường (anomalies) không: qty=0 là không hợp lệ

**Phương pháp: Value Distribution Analysis**
- Đếm frequency của từng giá trị qty (0, 1, 2, 3,...)
- Tính tỷ lệ % (proportion)
- Visualize bằng bar chart

In [ ]:
# --- Đoạn này để tạo 1 cái bảng thống kê cột "qty" ---

# 1. tính tỷ lệ % của các giá trị trong cột "qty" (ví dụ: số 0 chiếm 25.83%)
qty_prop = round(df["qty"].value_counts(normalize = True)*100,2)

# 2. đổi tên series, sắp xếp (sort) theo index (0, 1, 2...) và chuyển thành dataframe
qty_prop = qty_prop.rename("Proportion(%)").sort_index().to_frame()

# 3. đổi tên index (chỉ mục) thành "qty_value"
qty_prop.index.name = "qty_value"

# 4. thêm cột "Count" (Số đếm)
qty_prop["Count"] = df["qty"].value_counts()

# 5. đổi kiểu dữ liệu của cột "Proportion(%)" sang chuỗi (str) và thêm dấu "%" vào
qty_prop["Proportion(%)"] = (
    qty_prop["Proportion(%)"].astype(str)
    .str.cat(np.full(fill_value = "%",shape = (len(qty_prop),)))
)

# In cái bảng thống kê đó ra
qty_prop

In [ ]:
# ==============================================================
# VẼ BIỂU ĐỒ PHÂN PHỐI CỘT "QTY"
# ==============================================================

# Tạo figure
fig, ax = plt.subplots(figsize=(10, 5))

# Vẽ bar chart (trục Y = Count)
qty_prop.plot(kind="bar", y="Count", ax=ax)

# ️ Tiêu đề và nhãn
plt.xlabel("Số lượng (Qty) mỗi đơn", fontsize=13)
plt.ylabel("Tần suất [Số đếm]", fontsize=13)
plt.yticks(ticks=range(20000, 60001, 20000))
plt.title('Phân phối của cột "qty"', fontsize=14, fontweight=550)
plt.xticks(rotation="horizontal")
plt.legend("")  # Tắt legend

#  Thêm nhãn % lên mỗi cột
for x, y in enumerate(qty_prop["Count"]):
    qty_value = qty_prop.index[x]
    plt.text(x, y+1500, qty_prop["Proportion(%)"].loc[qty_value],
             horizontalalignment="center", fontweight=700)

# Thêm GHI CHÚ ĐẶC BIỆT cho qty=0
plt.text(0, 22000, 
         ' Giá trị này\n được coi là\n "NaN" (Lỗi)\n vì Số lượng\n không thể \n là "0"\n↓',
         horizontalalignment="center", color="darkred", fontsize=13)

plt.show()

#### **NHẬN XÉT VỀ CỘT "QTY":**

**Quan sát từ biểu đồ:**

Phân phối cột qty cho thấy 73.3% đơn hàng có qty=1, trong khi 26.46% có qty=0 (bất thường vì đơn hàng không thể có số lượng bằng 0), và chỉ có 0.24% có qty ≥ 2.  Giá trị qty=0 có thể do data entry error, system glitch, hoặc đơn hàng bị cancelled trước khi xác nhận nhưng vẫn được ghi log vào hệ thống.

**Vấn đề phát hiện:**
- Cột `qty` chứa giá trị 0 (đơn hàng bất thường)
- Sau phân tích, phát hiện `qty=0` có correlation quá cao với target (potential perfect predictor)
- **Quyết định:** Xóa toàn bộ cột `qty` để tránh model phụ thuộc vào anomaly data

**Lý do loại bỏ:**
1. **Tránh overfitting:** Feature có correlation quá cao (>0.5) với target có thể gây overfitting
2. **Generalization:** Model không nên phụ thuộc vào edge cases (qty=0)
3. **Business logic:** Trong thực tế, các đơn hàng qty=0 thường được filter sớm hơn trong pipeline

**Phương pháp:**
- Xóa trực tiếp cột `qty` khỏi DataFrame
- Không tạo feature mới từ qty


In [ ]:
# ==============================================================
# XỬ LÝ CỘT 'qty' (Số lượng)
# ==============================================================

print("="*70)
print("XỬ LÝ CỘT 'qty'")
print("="*70)

# Phân tích: qty có giá trị 0 (bất thường)
print("\nKiểm tra phân phối qty:")
print(df['qty'].value_counts().head())
print(f"\nSố lượng qty=0: {(df['qty'] == 0).sum()} ({(df['qty'] == 0).sum() / len(df) * 100:.2f}%)")

# Quyết định: Xóa cột qty
# Lý do: 
# 1. qty=0 là edge case, có thể gây overfitting
# 2.  Correlation với target quá cao (potential perfect predictor)
# 3. Model nên học từ features "bình thường" hơn

print("\nĐang xóa cột 'qty'...")
df. drop("qty", axis="columns", inplace=True)
print(" Đã xóa cột 'qty'")

print(f"\nDataFrame sau khi xử lý:")
print(f"  - Số dòng: {len(df)}")
print(f"  - Số cột: {len(df.columns)}")

print("\n Hoàn thành xử lý qty!")


<a id="amount"></a>
### <br>**4.b.vii. Phân tích cột "amount" (Số tiền).**

**Mục đích:** 
- Kiểm tra phân phối của số tiền đơn hàng
- Xác định và xử lý outliers (giá trị ngoại lệ)

**Phương pháp: Log Transformation**

Sử dụng Log Transformation để chuẩn hóa phân phối thay vì loại bỏ outliers. Phương pháp này cho phép giữ nguyên 100% dữ liệu trong khi vẫn giảm skewness.  Các giá trị amount cao (5000 Rs) là đơn hàng đắt tiền hợp lệ (saree sets, premium products), không phải noise hay errors, nên cần preserve để model có thể học được patterns ở mọi price ranges.

In [ ]:
# ==============================================================
# KIỂM TRA GIÁ TRỊ amount = 0
# ==============================================================

#  Đếm số dòng có amount = 0
amount0_count = (df["amount"] == 0).sum()

print(f"🔍 Kiểm tra giá trị amount = 0...")
print(f"   → Tìm thấy {amount0_count} dòng có amount = 0")
print(f"   → Tỷ lệ: {amount0_count/len(df)*100:.2f}%")

# Giải thích: Đơn hàng không thể có amount = 0
# → Đây là missing data hoặc lỗi nhập liệu (data entry error)

In [ ]:
# ==============================================================
# XỬ LÝ amount = 0 - XÓA DÒNG (Không tạo NaN)
# ==============================================================

print("\n Xử lý các dòng có amount = 0...")

# Đếm trước khi xóa
before_drop = len(df)

# Xóa các dòng có amount = 0
df = df[df["amount"] != 0]

# Đếm sau khi xóa
after_drop = len(df)
dropped = before_drop - after_drop

print(f"   → Đã xóa {dropped} dòng có amount = 0")
print(f"   → Còn lại: {after_drop} dòng")

# Kiểm tra: Không còn NaN
print(f" Không có giá trị NaN (NaN count: {df['amount'].isna().sum()})")

<br>**Kiểm tra Phân phối của cột "amount"**


In [ ]:
# ==============================================================
# VẼ BIỂU ĐỒ PHÂN PHỐI "AMOUNT" (Trước khi áp dụng Log)
# ==============================================================

#  Tạo 2 biểu đồ: Boxplot + Histogram
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(7, 8))

#  Tiêu đề chung
plt.suptitle("\nPhân phối của cột Amount (Số tiền) - Dữ liệu gốc", fontweight=600)

#  Biểu đồ 1: BOXPLOT (Biểu đồ hộp)
df["amount"].plot(kind="box", vert=False, ax=ax1)
ax1.text(3000, 1.15, "<-- Giá trị Ngoại lệ (Outliers) -->", 
         fontsize=17, horizontalalignment="center", color="darkred")
ax1.set_ylabel('Cột "Amount"', fontsize=12, fontweight=600)
ax1.set_yticklabels(labels="")

#  Biểu đồ 2: HISTOGRAM (Biểu đồ tần suất)
df["amount"].plot(kind="hist", bins=20, ax=ax2)
ax2.text(3000, 5000, "<-- Giá trị Ngoại lệ (Outliers) -->", 
         fontsize=17, horizontalalignment="center", color="darkred")
ax2.set_ylabel('Tần suất [Số đếm]', fontsize=12, fontweight=600)
ax2.set_xlabel("Số tiền (Rs.)", fontsize=12, fontweight=600)

plt.show()

#### NHẬN XÉT:

**Quan sát từ biểu đồ:**

Phân phối amount rất lệch phải (right-skewed) với nhiều outliers ở giá trị cao (>2000 Rs).  Phần lớn đơn hàng tập trung ở khoảng 200-800 Rs, trong khi có một số đơn hàng đắt tiền (expensive orders) lên đến 5000-10000 Rs.

**Quyết định - Áp dụng Log Transformation:**

Chúng ta sử dụng Log Transformation (`log(amount + 1)`) thay vì loại bỏ outliers vì các lý do sau:

**Lý do 1 - Preserve Information:** Các giá trị amount cao (5000 Rs) là đơn hàng đắt tiền hợp lệ (saree sets, premium products), không phải errors.   Nếu loại bỏ chúng, model sẽ mất thông tin quan trọng về việc "high-value orders có rejection rate khác thế nào so với low-value orders".

**Lý do 2 - Normalize Distribution:** Log transformation giảm skewness, làm cho distribution gần với Normal distribution hơn, giúp model học tốt hơn (đặc biệt với các thuật toán giả định Normal distribution như Logistic Regression

<br>**Kiểm tra phân phối mới của "amount" (sau khi bỏ outliers)** 


In [ ]:
# ==============================================================
# XỬ LÝ VÀ LOG TRANSFORM CỘT "AMOUNT" - AN TOÀN NHẤT
# ==============================================================

print(" Xử lý và transform cột 'amount'...")

# BƯỚC 1: Chuyển sang numeric
print("\n Chuyển đổi dtype...")
df['amount'] = pd.to_numeric(df['amount'], errors='coerce')
print(f"    Dtype: {df['amount'].dtype}")

# BƯỚC 2: Xóa NaN
print("\n Xử lý NaN...")
nan_before = df['amount'].isna().sum()
df = df.dropna(subset=['amount'])
nan_after = df['amount'].isna().sum()
print(f"   → Đã xóa {nan_before - nan_after} NaN values")

# BƯỚC 3: Xóa giá trị <= 0
print("\n Xử lý giá trị không hợp lệ...")
zero_negative = (df['amount'] <= 0).sum()
if zero_negative > 0:
    print(f"   → Tìm thấy {zero_negative} giá trị <= 0")
    df = df[df['amount'] > 0]
    print(f"    Đã xóa.  Còn lại: {len(df)} dòng")

# BƯỚC 4: Thống kê trước transform
print("\n Thống kê TRƯỚC transform:")
print(df['amount'].describe())

# BƯỚC 5: Log transformation
print("\n Áp dụng Log Transformation...")
df['amount_log'] = np.log1p(df['amount'])
print("    Đã tạo 'amount_log'")

# BƯỚC 6: Thống kê sau transform
print("\n Thống kê SAU transform:")
print(df['amount_log'].describe())

# BƯỚC 7: Reset index
df = df.reset_index(drop=True)

print("\n HOÀN THÀNH!")
print(f"📋 Dataset cuối: {len(df)} dòng, {len(df.columns)} cột")

In [ ]:
# ==============================================================
# VẼ BIỂU ĐỒ SO SÁNH: Original vs Log Transformed
# ==============================================================

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Biểu đồ 1: Dữ liệu gốc (có outliers, skewed)
df["amount"].dropna().plot(kind="hist", bins=30, ax=ax1, color="lightcoral")
ax1.set_title("Phân phối Amount Gốc\n(Có outliers, lệch phải)", 
              fontsize=12, fontweight=600)
ax1.set_xlabel("Số tiền (Rs.)")
ax1.set_ylabel("Tần suất")

#  Biểu đồ 2: Sau Log transformation (normalized)
df["amount_log"].dropna().plot(kind="hist", bins=30, ax=ax2, color="lightgreen")
ax2.set_title("Phân phối Amount sau Log Transform\n(Đã chuẩn hóa, phù hợp ML)", 
              fontsize=12, fontweight=600)
ax2.set_xlabel("Log(Amount + 1)")
ax2.set_ylabel("Tần suất")

plt.tight_layout()
plt.show()

# So sánh độ lệch (skewness)
print("\n So sánh độ lệch (Skewness):")
print(f"   Dữ liệu gốc: {df['amount'].skew():.2f} (rất lệch)")
print(f"   Sau Log transform: {df['amount_log'].skew():.2f} (ít lệch hơn)")

# Reset index (vì đã xóa một số dòng)
df = df.reset_index(drop=True)

print("\n Hoàn thành xử lý cột 'amount'!")
print(f"Dataset hiện có: {len(df)} dòng, {len(df.columns)} cột")

### **4.b.viii.  Phân tích cột "fulfilment" (Nguồn thực hiện đơn hàng)**

**Mục đích:** Kiểm tra phân phối của 2 loại fulfilment: Amazon (FBA) vs Merchant (FBM)

**Phương pháp:** Value counts + proportion analysis

In [ ]:
# tạo dataframe để kiểm tra tỷ lệ % của 2 loại "fulfilment" (Merchant vs Amazon)
df_cat = df["fulfilment"].value_counts().to_frame()
# Thêm cột tỷ lệ %
df_cat["proportion"] = round(df["fulfilment"].value_counts(normalize = True)*100,2)
# Thêm dấu "%" vào
df_cat["proportion"] = df_cat["proportion"].astype(str).str.cat(np.full((2,),"%"))
# Đổi tên các cột cho đẹp
df_cat = (df_cat.reset_index()
          .rename(columns = {"fulfilment":"count","index":"fulfilment_type"}))
# In bảng ra
df_cat

#### NHẬN XÉT:

**Quan sát:** Phân phối lệch (Merchant ~86%, Amazon ~13%) nhưng cả 2 nhóm đều có số lượng đủ lớn (không quá nhỏ) nên vẫn tốt cho model học.

**Quyết định:** Giữ nguyên, không xử lý gì thêm. 

### **4.b.ix. Phân tích cột "saleschannel" (Kênh bán hàng)**

**Mục đích:** Kiểm tra có bao nhiêu sales channels

In [ ]:
# ==============================================================
# PHÂN TÍCH CỘT "SALESCHANNEL"
# ==============================================================

# Tạo dataframe để kiểm tra tỷ lệ %
df_cat = df["saleschannel"].value_counts().to_frame()

# Thêm cột tỷ lệ %
df_cat["proportion"] = round(df["saleschannel"].value_counts(normalize=True) * 100, 4)

num_values = len(df_cat)  # Số lượng unique values thực tế

# Thêm dấu "%" vào (dùng số lượng thực tế)
df_cat["proportion"] = df_cat["proportion"].astype(str).str.cat(np.full((num_values,), "%"))

# Đổi tên các cột cho đẹp
df_cat = df_cat.reset_index().rename(columns={"saleschannel": "count", "index": "saleschannel_type"})

# In bảng ra
print(" Phân phối Saleschannel:")
print(df_cat)

#### ** NHẬN XÉT:**

**Quan sát:** 

Sau quá trình làm sạch dữ liệu (xóa amount=0, qty=0, status không rõ ràng), cột "saleschannel" chỉ còn lại **100% là "Amazon.  in"** (35,265 dòng).    Dòng duy nhất "Non-Amazon" trong raw data đã bị loại bỏ trong các bước tiền xử lý trước đó.

**Phân tích:**

Cột này không còn cung cấp thông tin hữu ích (zero variance - không có sự khác biệt giữa các dòng).   Trong machine learning, features với zero variance không giúp model phân biệt được giữa các classes vì tất cả samples đều có cùng một giá trị.
-->Xóa cột

In [ ]:
# Xóa (drop) cột "saleschannel"
df.drop("saleschannel", axis = "columns", inplace = True)

### **4.b.x. Phân tích cột "shipservicelevel" (Mức độ dịch vụ vận chuyển)**

**Mục đích:** Kiểm tra phân phối Standard vs Expedited shipping

In [ ]:
# tạo dataframe để kiểm tra tỷ lệ % (Standard vs Expedited)
df_cat = df["shipservicelevel"].value_counts().to_frame()
# Thêm cột tỷ lệ %
df_cat["proportion"] = round(df["shipservicelevel"].value_counts(normalize = True)*100,2)
# Thêm dấu "%" vào
df_cat["proportion"] = df_cat["proportion"].astype(str).str.cat(np.full((2,),"%"))
# Đổi tên các cột cho đẹp
df_cat = (df_cat.reset_index()
          .rename(columns = {"shipservicelevel":"count","index":"shipservicelevel_type"}))
# In bảng ra
df_cat

#### NHẬN XÉT:

**Quan sát:** Lệch (Standard nhiều hơn) nhưng cả 2 nhóm đều có số lượng đủ lớn. 

**Quyết định:** Giữ nguyên

### **4.b. xi. Phân tích cột "category" (Loại sản phẩm)**

**Mục đích:** Kiểm tra phân phối các loại sản phẩm (kurta, set, saree, etc.)

**Phương pháp:** Bar chart visualization

In [ ]:
# ----- BẮT ĐẦU VẼ BIỂU ĐỒ -----
fig,ax = plt.subplots(figsize = (7,5))

# tạo dataframe để kiểm tra tỷ lệ % của các loại 'category'
df_cat = df["category"].value_counts().to_frame()
# Thêm cột tỷ lệ %
df_cat["proportion"] = round(df["category"].value_counts(normalize = True)*100,2)
# Thêm cột nhãn % (để lát vẽ lên biểu đồ)
df_cat["proportion_label"] = df_cat["proportion"].astype(str).str.cat(np.full((len(df_cat),),"%"))

# Đổi tên cột 'index' (tên cũ) thành 'category_type'
# (Tớ đã sửa lại code ở đây cho chuẩn)
df_cat = df_cat.reset_index().rename(columns = {"category":"category_type", "count":"count"})

# Vẽ biểu đồ cột (bar), X là 'category_type', Y là 'proportion'
df_cat.plot(
    kind = "bar", x = "category_type",y = "proportion",
    legend = "", ax = ax
)
# Đặt Tiêu đề và Nhãn 
plt.xlabel("Loại Hàng")
plt.ylabel("Tỷ lệ (%)")
plt.title('Phân phối của cột "category"', fontsize = 14)
plt.xticks(rotation = 35) # Xoay chữ trục X 35 độ
plt.yticks(ticks = range(10,51,10) ,labels =["10%","20%","30%","40%","50%"])

# Thêm nhãn số đếm (Count) lên trên mỗi cột
for x,y in enumerate(df_cat["count"]):
    plt.text(x,1+df_cat["proportion"][x],y,
             fontsize = 10, horizontalalignment = "center")
plt.show()

#### NHẬN XÉT:

**Quan sát:** Phân phối lệch (Set, Kurta, Western Dress là chính) nhưng tất cả categories đều có số lượng đủ để model học.

**Quyết định:** Giữ nguyên

### **4.b.xii. Phân tích cột "b2b" (Business-to-Business)**

**Mục đích:** Kiểm tra tỷ lệ đơn hàng B2B (bán cho doanh nghiệp) vs B2C (bán cho cá nhân)

In [ ]:
# tạo dataframe để kiểm tra tỷ lệ % (True vs False)
df_cat = df["b2b"].value_counts().to_frame()
# Thêm cột tỷ lệ %
df_cat["proportion"] = round(df["b2b"].value_counts(normalize = True)*100,2)
# Thêm dấu "%" vào
df_cat["proportion"] = df_cat["proportion"].astype(str).str.cat(np.full((2,),"%"))
# Đổi tên các cột cho đẹp
df_cat = (df_cat.reset_index()
          .rename(columns = {"b2b":"count","index":"b2b_type"}))
# In bảng ra
df_cat

#### **NHẬN XÉT:**

**Quan sát:** 99.35% là B2C (False), chỉ có 303 đơn B2B (True - 0.65%).  Tuy lệch nhưng nhóm thiểu số vẫn có 303 samples (không quá nhỏ). 

**Quyết định:** 
-  Giữ lại feature này
-  Chuyển dtype từ `bool` → `object` để model hiểu đây là categorical (2 loại), không phải số 0/1

In [ ]:
# đổi kiểu dữ liệu 'b2b' (hiện đang là bool True/False) sang 'object' (chuỗi)
# Model sẽ hiểu đây là 2 "loại" (True/False) chứ không phải số (1/0)
df["b2b"] = df["b2b"].astype(object)

### **4.b.xiii. Phân tích cột "shipstate" (Bang giao hàng)**

**Mục đích:** 
- Kiểm tra phân phối 37 bang của Ấn Độ
- Tạo feature mới "region" (vùng địa lý) để giảm cardinality

**Phương pháp: Feature Engineering - Grouping**

Ấn Độ có 28 states + 8 union territories + 1 capital = 37 administrative divisions, nhưng có thể gom thành 6 vùng địa lý chính: North India, South India, East India, West India, Central India, và Northeast India.  


In [ ]:
# tạo dataframe để kiểm tra tỷ lệ % của các 'shipstate'
df_cat = df["shipstate"].value_counts().to_frame()
# Thêm cột tỷ lệ %
df_cat["proportion"] = round(df["shipstate"].value_counts(normalize = True)*100,2)
# Thêm cột nhãn % (dạng chuỗi)
df_cat["proportion_str"] = df_cat["proportion"].astype(str).str.cat(np.full((len(df_cat),),"%"))

# Đổi tên cột 'index' (tên cũ) thành 'shipstate_type'
# (Tớ đã sửa lại code ở đây cho chuẩn)
df_cat = (df_cat.reset_index()
          .rename(columns = {"shipstate":"shipstate_type", "count":"count"}))

# ----- BẮT ĐẦU VẼ BIỂU ĐỒ -----
# Sắp xếp các bang (từ ít -> nhiều) để vẽ biểu đồ ngang (barh) cho đẹp
df_cat = df_cat.sort_values(by = "proportion").reset_index(drop = True)
fig,ax = plt.subplots(figsize = (7,10)) # Biểu đồ cao 10 inch

# Vẽ biểu đồ ngang (barh), Y là 'shipstate_type', X là 'proportion'
df_cat.plot(
    kind = "barh", x = "shipstate_type",y = "proportion",
    legend = "", ax = ax
)
# Đặt Tiêu đề và Nhãn (ĐÃ DỊCH)
plt.ylabel("Bang giao hàng")
plt.xlabel("Tỷ lệ (%)")
plt.title('Phân phối của cột "shipstate"', fontsize = 14)
# Hiển thị tất cả 37 tên bang trên trục Y
plt.yticks(ticks = list(range(0,len(df_cat))) ,labels=df_cat["shipstate_type"],
          fontsize = 10)
plt.xticks(ticks = range(5,26,5) ,labels =["5%","10%","15%","20%","25%"])

# Thêm nhãn số đếm (Count) vào cuối mỗi thanh ngang
for x,y in enumerate(df_cat["count"]):
    plt.text((df_cat["proportion"][x]),x,y,
             fontsize = 10, verticalalignment = "center")
plt.show()

####  NHẬN XÉT A: Phân phối ban đầu (37 states)**

**Quan sát:** 

Phân phối rất lệch với Maharashtra và Karnataka chiếm phần lớn.  Một số bang Đông-Bắc (Northeast states) và Union Territories có số lượng rất nhỏ (dưới 100 orders), khiến model khó học patterns cho những bang này.  Ngoài ra, có 13 dòng "UNKNOWN" (không xác định được bang). 

**Quyết định:**
1. Xóa 13 dòng "UNKNOWN" (chiếm tỷ lệ nhỏ)
2. Tạo feature mới "region" - gom 37 bang thành 6 vùng địa lý
3. Xóa cột "shipstate" gốc sau khi có "region"

In [ ]:
# ==============================================================
# TẠO FEATURE MỚI: "REGION" (Vùng địa lý)
# ==============================================================
# Feature Engineering: Gom 37 bang → 6 vùng
# - Giảm cardinality: 37 → 6
# - Giữ được thông tin địa lý
# - Model dễ học hơn (6 categories thay vì 37)

print("Tạo feature: Region")

df["region"] = df["shipstate"].replace({
    # WEST INDIA (Tây Ấn)
    "MAHARASHTRA": "westindia", "GUJARAT": "westindia",
    "RAJASTHAN": "westindia", "GOA": "westindia",
    "DADRA AND NAGAR": "westindia",
    
    # SOUTH INDIA (Nam Ấn)
    "KARNATAKA": "southindia", "TAMIL NADU": "southindia",
    "KERALA": "southindia", "ANDHRA PRADESH": "southindia",
    "TELANGANA": "southindia", "PUDUCHERRY": "southindia",
    "LAKSHADWEEP": "southindia",
    
    # NORTH INDIA (Bắc Ấn)
    "UTTAR PRADESH": "northindia", "HARYANA": "northindia",
    "PUNJAB": "northindia", "UTTARAKHAND": "northindia",
    "HIMACHAL PRADESH": "northindia", "JAMMU & KASHMIR": "northindia",
    "CHANDIGARH": "northindia", "LADAKH": "northindia",
    
    # EAST INDIA (Đông Ấn)
    "WEST BENGAL": "eastindia", "JHARKHAND": "eastindia",
    "ODISHA": "eastindia", "BIHAR": "eastindia",
    "CHHATTISGARH": "eastindia", "ANDAMAN & NICOBAR ": "eastindia",
    
    # CENTRAL INDIA (Trung Ấn)
    "MADHYA PRADESH": "centralindia", "NEW DELHI": "centralindia",
    
    # NORTHEAST INDIA (Đông Bắc Ấn)
    "ASSAM": "northeastindia", "MEGHALAYA": "northeastindia",
    "TRIPURA": "northeastindia", "SIKKIM": "northeastindia",
    "MANIPUR": "northeastindia", "MIZORAM": "northeastindia",
    "NAGALAND": "northeastindia", "ARUNACHAL PRADESH": "northeastindia"
})

# Xóa các dòng "UNKNOWN"
before_drop = len(df)
df = df[df["shipstate"] != "UNKNOWN"]
after_drop = len(df)

In [ ]:
# ==============================================================
# VẼ BIỂU ĐỒ: Phân phối "REGION" (6 vùng)
# ==============================================================

df_cat = df["region"].value_counts().to_frame()
df_cat["proportion"] = round(df["region"].value_counts(normalize=True)*100, 2)
df_cat["proportion_str"] = df_cat["proportion"].astype(str).str.cat(np.full((len(df_cat),), "%"))
df_cat = df_cat.reset_index().rename(columns={"region":"region_name", "count":"count"})
df_cat = df_cat.sort_values(by="proportion").reset_index(drop=True)

fig, ax = plt.subplots(figsize=(7, 5))
df_cat.plot(kind="barh", x="region_name", y="proportion", legend="", ax=ax)

plt.ylabel("Vùng")
plt.xlabel("Tỷ lệ (%)")
plt.title('Phân phối của cột "region"', fontsize=14)
plt.yticks(ticks=list(range(0, len(df_cat))), labels=df_cat["region_name"])
plt.xticks(ticks=range(10, 51, 10), labels=["10%", "20%", "30%", "40%", "50%"])

for x, y in enumerate(df_cat["count"]):
    plt.text((df_cat["proportion"][x]), x, y, fontsize=10, verticalalignment="center")

plt.show()

#### **NHẬN XÉT B: Sau khi tạo "region"**

**Quan sát:** 

Phân phối 6 vùng vẫn lệch (South India và West India chiếm phần lớn) nhưng TẤT CẢ các vùng đều có số lượng đủ lớn (ngay cả Northeast India có số lượng thấp nhất cũng vẫn có vài trăm orders). Điều này tốt hơn nhiều so với việc có 37 categories với một số bang chỉ có vài chục orders.

**Kết quả:**
-  Giảm cardinality: 37 states → 6 regions
-  Tất cả regions đều có sample size đủ lớn để model học
-  Vẫn giữ được thông tin địa lý quan trọng

**Quyết định cuối:** 
- Giữ "region" 
- Xóa "shipstate" (redundant)

In [ ]:
# Xóa (drop) cột "shipstate" gốc (vì đã có cột "region" thay thế)
df.drop("shipstate",axis = "columns", inplace = True)

# reset (đặt lại) chỉ mục (index) của dataframe sau khi đã xóa nhiều dòng
df = df.reset_index(drop = True)
# Kiểm tra lại thông tin lần cuối, xem còn bao nhiêu cột
df.info()

<a id="final_wrangle"></a>
## 4.c. FINAL WRANGLE FUNCTION
**Mục đích:** 
Tổng hợp TẤT CẢ các bước làm sạch đã phân tích ở EDA vào một hàm duy nhất `final_wrangle(filepath)` để có thể tái sử dụng. 

**Phương pháp: ETL Pipeline**
- **Extract:** Đọc file CSV
- **Transform:** Áp dụng tất cả cleaning rules
- **Load:** Return DataFrame sạch, sẵn sàng cho modeling

In [ ]:
# ==============================================================
# HÀM FINAL_WRANGLE (Xử lý và Làm sạch Dữ liệu Tổng hợp)
# ==============================================================

def final_wrangle(filepath):
    # BƯỚC 1: Đọc file
    print(" Đang tiến hành đọc dữ liệu từ tập tin...")
    df = pd.read_csv(filepath, low_memory=False)
    print(f"   → Đã tải thành công {len(df)} dòng và {len(df.columns)} cột.")
    
    # BƯỚC 2: Chuẩn hóa tên cột (Clean column names)
    col = [element.lower().replace(" ", "").replace("-", "") for element in df.columns]
    df.columns = col
    
    # BƯỚC 3: Loại bỏ các thuộc tính dư thừa (Drop redundant columns)
    df.drop(["index", "date", "fulfilledby", "currency", "unnamed:22",
             "promotionids", "courierstatus", "shipcountry"],
            axis="columns", inplace=True)
    
    # BƯỚC 4: Xử lý giá trị thiếu cho thông tin địa lý (Fill NaN for location)
    df["shipstate"].fillna("unknown", inplace=True)
    df["shipcity"].fillna("unknown", inplace=True)
    df["shippostalcode"].fillna(0, inplace=True)
    df["shippostalcode"] = df["shippostalcode"].astype(int).astype(object)
    
    # BƯỚC 5: Chuẩn hóa tên bang (Standardize shipstate)
    df["shipstate"] = df["shipstate"].str.upper()
    df["shipstate"].replace({
        "PONDICHERRY": "PUDUCHERRY", "RAJSHTHAN": "RAJASTHAN", "RAJSTHAN": "RAJASTHAN",
        "RJ": "RAJASTHAN", "PB": "PUNJAB", "PUNJAB/MOHALI/ZIRAKPUR": "PUNJAB",
        "ORISSA": "ODISHA", "DELHI": "NEW DELHI", "NL": "UNKNOWN", "APO": "UNKNOWN", "AR": "UNKNOWN"
    }, inplace=True)
    
    # BƯỚC 6: Loại bỏ các bản ghi trùng lặp (Drop duplicates)
    df.drop_duplicates(inplace=True)
    
    # BƯỚC 7-9: Thu hẹp phạm vi nghiên cứu: Chỉ giữ đơn hàng đơn lẻ (Remove multi-product orders)
    df = df[df["orderid"].duplicated(keep=False) == False]
    df.drop("orderid", axis=1, inplace=True)
    
    # BƯỚC 10: Thiết lập biến mục tiêu "rejected" (Create target variable)
    known_value = ["Cancelled", 'Shipped - Returned to Seller', 'Shipped - Rejected by Buyer',
                   'Shipped - Returning to Seller', 'Shipped - Delivered to Buyer']
    df = df[df["status"].isin(known_value)]
    
    rejected = ["Cancelled", 'Shipped - Returned to Seller', 'Shipped - Rejected by Buyer',
                'Shipped - Returning to Seller']
    df["rejected"] = df["status"].isin(rejected).astype(int)
    df.drop("status", axis="columns", inplace=True)
    
    # BƯỚC 11: Loại bỏ đặc trưng có số chiều cao (Drop high cardinality features)
    df.drop(["style", "sku", "shipcity", "shippostalcode", "asin"], axis=1, inplace=True)
    
    # BƯỚC 12: XỬ LÝ SỐ LƯỢNG (QTY) → Trích xuất đặc trưng nhị phân
    print("\n Đang xử lý thuộc tính 'qty' (Số lượng)...")
    
    df.drop("qty", axis="columns", inplace=True)
    print("   → Đã xóa cột 'qty'")
    
    # BƯỚC 13: XỬ LÝ GIÁ TRỊ ĐƠN HÀNG (AMOUNT) → Biến đổi Logarit
    print("\n Đang xử lý thuộc tính 'amount' (Giá trị đơn hàng)...")
    
    # 13.1.  CHUYỂN ĐỔI DTYPE SANG NUMERIC (QUAN TRỌNG!)
    print("   → Chuyển đổi dtype sang numeric...")
    df["amount"] = pd.to_numeric(df["amount"], errors='coerce')
    print(f"   → Dtype hiện tại: {df['amount'].dtype}")
    
    # 13.2.  Kiểm tra và xử lý NaN
    nan_count = df["amount"].isna().sum()
    if nan_count > 0:
        print(f"   → Tìm thấy {nan_count} giá trị NaN, đang xóa...")
        df = df.dropna(subset=["amount"])
        print(f"   → Đã xóa {nan_count} dòng có NaN")
    
    # 13.3. Xóa các dòng có amount = 0
    amount_zero_count = (df["amount"] == 0).sum()
    if amount_zero_count > 0:
        print(f"   → Tìm thấy {amount_zero_count} dòng có amount=0")
        df = df[df["amount"] != 0]
        print(f"   → Đã xóa {amount_zero_count} dòng")
    
    # 13. 4. Xóa các giá trị âm (nếu có)
    negative_count = (df["amount"] < 0).sum()
    if negative_count > 0:
        print(f"   → Tìm thấy {negative_count} giá trị âm, đang xóa...")
        df = df[df["amount"] > 0]
    
    # 13.5.  ÁP DỤNG LOG TRANSFORMATION (Bây giờ sẽ hoạt động!)
    print("   → Áp dụng Log Transformation...")
    df["amount_log"] = np.log1p(df["amount"])
    print("   Đã tạo feature 'amount_log'")
    
    print(f"   → Số dòng còn lại: {len(df)}")
    
    # BƯỚC 14: Loại bỏ "saleschannel"
    df.drop("saleschannel", axis="columns", inplace=True)
    
    # BƯỚC 15: Tạo đặc trưng vùng miền "region" (Feature Engineering)
    print("\n Đang thực hiện trích xuất đặc trưng 'region' (Vùng miền)...")
    df["region"] = df["shipstate"].replace({
        "MAHARASHTRA": "westindia", "KARNATAKA": "southindia",
        "PUDUCHERRY": "southindia", "TELANGANA": "southindia",
        "ANDHRA PRADESH": "southindia", "HARYANA": "northindia",
        "JHARKHAND": "eastindia", "CHHATTISGARH": "eastindia",
        "ASSAM": "northeastindia", "ODISHA": "eastindia",
        "UTTAR PRADESH": "northindia", "GUJARAT": "westindia",
        "TAMIL NADU": "southindia", "UTTARAKHAND": "northindia",
        "WEST BENGAL": "eastindia", "RAJASTHAN": "westindia",
        "NEW DELHI": "centralindia", "MADHYA PRADESH": "centralindia",
        "KERALA": "southindia", "JAMMU & KASHMIR": "northindia",
        "BIHAR": "eastindia", "MEGHALAYA": "northeastindia",
        "PUNJAB": "northindia", "GOA": "southindia",
        "TRIPURA": "northeastindia", "CHANDIGARH": "northindia",
        "HIMACHAL PRADESH": "northindia", "SIKKIM": "northeastindia",
        "ANDAMAN & NICOBAR ": "eastindia", "MANIPUR": "northeastindia",
        "MIZORAM": "northeastindia", "NAGALAND": "northeastindia",
        "ARUNACHAL PRADESH": "northeastindia", "LADAKH": "northindia",
        "DADRA AND NAGAR": "westindia", "LAKSHADWEEP": "southindia"
    })
    
    # Loại bỏ các bang không xác định
    df = df[df["shipstate"] != "UNKNOWN"]
    df.drop("shipstate", axis="columns", inplace=True)
    print("   → Đã tạo đặc trưng 'region' (gồm 6 vùng địa lý).")
    
    # BƯỚC 16: Chuyển đổi kiểu dữ liệu b2b
    df["b2b"] = df["b2b"].astype(object)
    
    # BƯỚC 17: Đặt lại chỉ mục
    df = df.reset_index(drop=True)
    
    # BƯỚC 18: KIỂM TRA CHẤT LƯỢNG DỮ LIỆU CUỐI CÙNG (FINAL DATA QUALITY CHECK)
    print("\n Kiểm tra chất lượng dữ liệu lần cuối...")
    nan_count = df.isna().sum()
    total_nans = nan_count.sum()
    
    if total_nans > 0:
        print("   CẢNH BÁO: Phát hiện giá trị thiếu (NaN) trong dữ liệu:")
        print(nan_count[nan_count > 0])
        raise ValueError("   LỖI: DataFrame chứa giá trị NaN! Quá trình SMOTE sẽ thất bại!")
    else:
        print("   Không phát hiện giá trị thiếu. Dữ liệu sẵn sàng cho SMOTE!")
    
    print(f"\n Bộ dữ liệu hoàn chỉnh: {len(df)} dòng, {len(df.columns)} cột.")
    print(f"   Danh sách cột: {df.columns.tolist()}")
    
    return df

# **5. DATA LOADING (Tải dữ liệu đã làm sạch)**

**Mục đích:** Áp dụng hàm `final_wrangle()` để có được dataset cuối cùng, sạch, sẵn sàng cho modeling

**Phương pháp: ETL Pipeline Execution**
- Execute toàn bộ cleaning pipeline một lần duy nhất
- Verify kết quả (shape, columns, no NaN)

In [ ]:
# 5. TẢI DỮ LIỆU - THỰC THI HÀM FINAL_WRANGLE()
filepath_list = glob("/kaggle/input/unlock-profits-with-e-commerce-sales-data/*.csv")
filepath = filepath_list[0]

print("ĐANG THỰC THI FINAL_WRANGLE() - QUY TRÌNH ETL TOÀN DIỆN")

# Chạy hàm làm sạch và xử lý dữ liệu
df = final_wrangle(filepath)


print(" QUÁ TRÌNH TẢI DỮ LIỆU HOÀN TẤT!")

#  Kiểm tra kết quả đầu ra
print("\n Thông tin Bộ dữ liệu Cuối cùng (Final Dataset Info):")
df.info()

print("\n 5 dòng dữ liệu đầu tiên:")
display(df.head())

# KIỂM TRA TÍNH HỢP LỆ (VALIDATION CHECK)
print("\n Các kiểm tra tính hợp lệ:")
print(f"    Kích thước (Shape): {df.shape}")
print(f"    Không có dữ liệu trùng lặp: {df.duplicated().sum() == 0}")
print(f"    Không có giá trị thiếu (NaN): {df.isna().sum().sum() == 0}")
print(f"    Biến mục tiêu 'rejected' tồn tại: {'rejected' in df.columns}")
print(f"    Phân phối biến mục tiêu: {df['rejected'].value_counts().to_dict()}")

print("\n Bộ dữ liệu đã sẵn sàng cho giai đoạn tiền xử lý và mô hình hóa!")

<a id = "data_preprocessing"></a>
# **6. DATA PREPROCESSING**

<a id = "split_dataset"></a>
## <br>**6.a. Chia Dataset**<br>

### **6.a.i.  Chia Dataset thành X (features) và y (target)**

**Mục đích:** Tách biến mục tiêu ("rejected") ra khỏi các features

**Phương pháp: Train-Test Split Preparation**

In [ ]:
# Cột "mục tiêu" (cái cần dự đoán)
target = "rejected"
y = df[target]     # y = vector mục tiêu (chỉ chứa cột "rejected")
X = df.drop(target, axis = "columns")   # X = ma trận đặc trưng 

# Hiển thị thông tin để kiểm tra

print("Split into X and y:")
print(f"   → y shape: {y.shape}")
print(f"   → X shape: {X.shape}")
print(f"\n Features (X columns): {X.columns.tolist()}")
print(f"\n Target distribution (y):")
print(y.value_counts())

### **6.a.ii. Chia thành Train và Test sets**

**Mục đích:** Chia data thành 80% train, 20% test

**Phương pháp: Stratified Split**

Sử dụng stratified split cho imbalanced data để đảm bảo tỷ lệ target classes giống nhau ở cả train và test sets.  Parameter `stratify=y` trong `train_test_split()` sẽ tự động maintain tỷ lệ 58:42 (Not Rejected:Rejected) ở cả 2 sets, tránh trường hợp test set có distribution khác train set (data distribution shift).

In [ ]:
# Phân chia dữ liệu có phân tầng (stratification)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y  # Thêm dòng này để cân bằng tỷ lệ lớp dữ liệu
)

print(" Hoàn tất phân chia tập Huấn luyện - Kiểm thử:")
print(f"   → X_train (Đặc trưng huấn luyện): {X_train.shape}")
print(f"   → X_test (Đặc trưng kiểm thử): {X_test.shape}")
print(f"   → y_train (Nhãn huấn luyện): {y_train.shape}")
print(f"   → y_test (Nhãn kiểm thử): {y_test.shape}")

# Kiểm tra phân phối lớp dữ liệu
print("\n Phân phối lớp trong tập HUẤN LUYỆN:")
print(y_train.value_counts(normalize=True))

print("\n Phân phối lớp trong tập KIỂM THỬ:")
print(y_test.value_counts(normalize=True))

print("\n Các phân phối tương đồng (Kỹ thuật phân tầng dữ liệu hoạt động hiệu quả!)")

## **6. b. Xử lý Imbalanced Data**

**Mục đích:** Cân bằng train set bằng SMOTE (chỉ oversample train, không động vào test)

**Phương pháp: SMOTENC (SMOTE for mixed data)**

SMOTENC là phiên bản mở rộng cho phép xử lý dữ liệu hỗn hợp (categorical + numerical).  
Quan trọng: chỉ áp dụng SMOTE trên train set để tránh data leakage (nếu oversample cả test set thì model sẽ "nhìn thấy" test data trong quá trình train).

### **6.b.i.  Kiểm tra sự mất cân bằng trong y_train**

**Mục đích:** Visualize phân phối target trong train set trước khi oversample

In [ ]:
# ==============================================================
# 6.b.i. KIỂM TRA IMBALANCE TRONG TRAIN SET
# ==============================================================

# Vẽ biểu đồ phân phối target trong train set (TRƯỚC SMOTE)
fig, ax = plt.subplots(figsize=(7, 5))

# Tính tỷ lệ % của 2 classes (0 và 1)
plot_dataseries = round(y_train.value_counts(normalize=True) * 100, 2)

# Vẽ biểu đồ cột
plot_dataseries.plot(kind="bar", ax=ax, color="navy")

# Thêm text cảnh báo
plt.text(0.5, 70, "Dataset Lớp Mất cân bằng", color="darkred",
         horizontalalignment="center", fontsize=14, fontweight=700)
plt.axhline(y=plot_dataseries[0], color="darkred", linestyle="--")

# Tiêu đề và nhãn
plt.title("Tỷ lệ các Lớp của y_train (Trước SMOTE)", fontsize=16)
plt.ylabel("Tỷ lệ (%)", fontsize=12)
plt.xlabel("Trạng thái", fontsize=12)
plt.xticks(ticks=range(len(plot_dataseries)),
           labels=["Không bị từ chối (0)", "Bị từ chối (1)"], 
           rotation="horizontal")
plt.yticks(ticks=[20, 40, 60, 80, 100], labels=["20%", "40%", "60%", "80%", "100%"])

# Thêm nhãn % lên cột
data_label = plot_dataseries.astype(str).str.cat(np.full((2,), "%"), sep="")
for x, y in enumerate(plot_dataseries):
    plt.text(x, y-7, data_label[x], color="white",
             fontweight=700, fontsize=13, horizontalalignment="center")

# Thêm nhãn số đếm (Count)
for x, y in enumerate(y_train.value_counts()):
    plt.text(x, plot_dataseries[x]-13, 
             f"Số lượng: {y_train.value_counts()[x]}",
             horizontalalignment="center", color="lightpink", 
             fontsize=12, fontweight=700)

plt.show()

# In kết quả ra console
print("Phân phối y_train (TRƯỚC SMOTE):")
print(y_train.value_counts())
print("\nTỷ lệ %:")
print(y_train.value_counts(normalize=True) * 100)

### **6.b.ii.  Áp dụng SMOTE để cân bằng train set**

**Mục đích:** Sử dụng SMOTENC để tạo synthetic samples cho minority class, đạt tỷ lệ 50-50

**SMOTE hoạt động như thế nào:**

SMOTE tạo synthetic samples mới bằng cách nội suy (interpolate) giữa các minority samples hiện có.  Cụ thể: 
(1) Chọn một minority sample, (2) Tìm k nearest neighbors của nó, (3) Chọn ngẫu nhiên một neighbor, (4) Tạo synthetic sample nằm trên đoạn thẳng nối giữa sample gốc và neighbor đó.  Điều này giúp model học được vùng decision boundary tốt hơn thay vì chỉ "ghi nhớ" các samples bị duplicate. 

In [ ]:
from imblearn.over_sampling import SMOTENC

print(" ÁP DỤNG SMOTENC (SMOTE for Mixed Categorical + Numerical Data)")
print()

# BƯỚC 1: XÁC ĐỊNH CATEGORICAL FEATURES
# SMOTENC cần biết cột nào là categorical để xử lý đúng
# → Lấy index (vị trí) của các cột categorical (dtype='object')

categorical_features = []
for i, col in enumerate(X_train.columns):
    if X_train[col].dtype == 'object':
        categorical_features.append(i)

print("Thông tin về features:")
print(f"   → Tổng số features: {len(X_train.columns)}")
print(f"   → Categorical feature indices: {categorical_features}")
print(f"   → Categorical features: {[X_train.columns[i] for i in categorical_features]}")
print(f"   → Numerical features: {list(X_train.select_dtypes(exclude='object').columns)}")
print()

# BƯỚC 2: KHỞI TẠO SMOTENC
#  Tạo SMOTENC instance
smote = SMOTENC(
    categorical_features=categorical_features,  # Chỉ định cột nào là categorical
    random_state=42,  # Để reproducible (chạy lại ra kết quả giống nhau)
    k_neighbors=5  # Số neighbors dùng để tạo synthetic samples
)
print(" SMOTENC Parameters:")
print(f"   → k_neighbors: 5 (mỗi synthetic sample tạo từ 5 neighbors gần nhất)")
print(f"   → random_state: 42 (đảm bảo reproducibility)")
print()
# BƯỚC 3: FIT-RESAMPLE (Chỉ trên TRAIN SET!)

print(" Đang áp dụng SMOTE trên train set...")
print(f"   → Train set TRƯỚC SMOTE: {X_train.shape}")
print(f"   → y_train TRƯỚC SMOTE: {y_train.value_counts().to_dict()}")
print()

#  Fit và resample
X_train_over, y_train_over = smote.fit_resample(X_train, y_train)

print(" SMOTE hoàn thành!")
print(f"   → Train set SAU SMOTE: {X_train_over.shape}")
print(f"   → y_train SAU SMOTE: {y_train_over.value_counts().to_dict()}")
print()

# BƯỚC 4: KIỂM TRA KẾT QUẢ

print(" Phân phối class sau SMOTE:")
print(y_train_over.value_counts())
print("\nTỷ lệ %:")
print(y_train_over.value_counts(normalize=True) * 100)
print()

# Kiểm tra cân bằng
class_0_pct = y_train_over.value_counts(normalize=True)[0] * 100
class_1_pct = y_train_over.value_counts(normalize=True)[1] * 100

if abs(class_0_pct - 50) < 0.1 and abs(class_1_pct - 50) < 0.1:
    print("Perfect balance: 50-50!")
else:
    print(f" Balance: {class_0_pct:.2f}% - {class_1_pct:.2f}%")

print()
print(" TRAIN SET ĐÃ ĐƯỢC CÂN BẰNG!")

In [ ]:
# ==============================================================
# VẼ BIỂU ĐỒ: TRAIN SET SAU KHI ÁP DỤNG SMOTE
# ==============================================================

fig, ax = plt.subplots(figsize=(8, 5))

# Tính tỷ lệ % của y_train_over (sau SMOTE)
plot_dataseries = round(y_train_over.value_counts(normalize=True) * 100, 2)

# Vẽ biểu đồ cột
plot_dataseries.plot(kind="bar", ax=ax, color="navy")

# Thêm text xác nhận cân bằng
plt.text(0.5, 70, "Dataset Lớp Cân bằng (Đã Oversample)", color="darkgreen",
         horizontalalignment="center", fontsize=14, fontweight=700)
plt.axhline(y=plot_dataseries[0], color="darkgreen", linestyle="--")

# Tiêu đề và nhãn
plt.title("Tỷ lệ các Lớp của y_train_over (Sau SMOTE)", fontsize=16)
plt.ylabel("Tỷ lệ (%)", fontsize=12)
plt.xlabel("Trạng thái", fontsize=12)
plt.xticks(ticks=range(len(plot_dataseries)),
           labels=["Không bị từ chối (0)", "Bị từ chối (1)"],
           rotation="horizontal")
plt.yticks(ticks=[20, 40, 60, 80, 100], labels=["20%", "40%", "60%", "80%", "100%"])

# Thêm nhãn % lên cột
data_label = plot_dataseries.astype(str).str.cat(np.full((2,), "%"), sep="")
for x, y in enumerate(plot_dataseries):
    plt.text(x, y-10, data_label[x], color="white",
             fontweight=700, fontsize=13, horizontalalignment="center")

# Thêm nhãn số đếm
for x, y in enumerate(y_train_over.value_counts()):
    plt.text(x, plot_dataseries[x]-25,
             f"Số lượng:\n{y_train_over.value_counts()[x]}",
             horizontalalignment="center", color="lightgreen", fontweight=700)

plt.show()

print("\n Biểu đồ đã hoàn thành!")

#### **KẾT QUẢ SAU KHI ÁP DỤNG SMOTE:**

**Train set đã được cân bằng:**
- **Trước SMOTE:** ~58% class 0, ~42% class 1 (lệch)
- **Sau SMOTE:** 50% class 0, 50% class 1 (cân bằng hoàn hảo)

**Lợi ích:**
1. **Giảm bias:** Model không bị thiên vị về class đa số
2. **Cải thiện Recall:** Model học được minority class tốt hơn
3. **Synthetic samples:** SMOTE tạo samples mới (không duplicate) → tránh overfitting
4. **Preserve test set:** Test set giữ nguyên distribution gốc → đánh giá công bằng


### **6.c. Data Transformation (Pipelines)**

**Mục đích:** 
- Xử lý NaN (nếu còn)
- Scale numerical features
- Encode categorical features

**Phương pháp: Sklearn Pipelines + ColumnTransformer**

Sklearn pipelines cho phép gói tất cả preprocessing steps vào một object duy nhất, đảm bảo cùng transformations được áp dụng đồng nhất cho train và test data (tránh data leakage).  ColumnTransformer cho phép áp dụng transforms khác nhau cho từng loại cột (numerical vs categorical).

### <br>**6.c.ii.** NUMERICAL PIPELINE.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Tạo pipeline: Điền giá trị thiếu bằng trung vị -> Chuẩn hóa
num_transformer = make_pipeline(
    SimpleImputer(strategy='median'),  # Trung vị ít bị ảnh hưởng bởi ngoại lai hơn trung bình
    StandardScaler()  # Công thức: (x - trung bình) / độ lệch chuẩn
)

print(" Numerical transformer pipeline created:")
num_transformer

### <br>**6.c.ii.** CATEGORICAL PIPELINE.

In [ ]:
# Pipeline: SimpleImputer (điền NaN) → OneHotEncoder (encode)
# - SimpleImputer với strategy='most_frequent': Điền bằng mode
# - OneHotEncoder với drop='first': Tránh dummy variable trap

cat_transformer = make_pipeline(
    SimpleImputer(strategy="most_frequent"),  # Điền NaN bằng mode
    OneHotEncoder(drop="first", handle_unknown='ignore')  # OneHot encode
)

print(" Categorical transformer created:")
print(cat_transformer)

### <br>**6.c.iii.** iii. ColumnTransformer (Bộ biến đổi cột) để chạy song song.


In [ ]:
# Cập nhật bộ biến đổi dữ liệu phân loại để xử lý các danh mục chưa xuất hiện
cat_transformer = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(drop="first", handle_unknown='ignore')  # Thêm tham số xử lý giá trị lạ
)

# Khởi tạo ColumnTransformer
col_transformer = ColumnTransformer([
    ("numtransformer", num_transformer,
     X_train_over.select_dtypes(exclude="object").columns),
    ("cattransformer", cat_transformer,
     X_train_over.select_dtypes(include="object").columns)
], remainder='drop')  # Loại bỏ các cột còn lại không được liệt kê

print("Column transformer created!")
print(f"\nNumerical features: {list(X_train_over.select_dtypes(exclude='object').columns)}")
print(f"Categorical features: {list(X_train_over.select_dtypes(include='object').columns)}")

col_transformer

# **7. XÂY DỰNG VÀ ĐÁNH GIÁ MÔ HÌNH MACHINE LEARNING**

## 7.1. IMPORT MODELS VÀ TẠO DATAFRAME LƯU KẾT QUẢ

In [ ]:

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, roc_auc_score, roc_curve, 
                             confusion_matrix, ConfusionMatrixDisplay, 
                             classification_report)

# Tạo dataframe lưu kết quả các models
results_df = pd.DataFrame(columns=[
    'Model', 'Baseline_Accuracy', 'Tuned_Accuracy', 
    'Precision', 'Recall', 'F1_Score', 'AUC_Score'
])

print("Bat dau xay dung cac model Machine Learning...")
print("DataFrame luu ket qua da duoc tao.\n")

## 7.2. Model: XGBoost


### Bước 1: BASELINE (Default Parameters)

In [ ]:
# ==============================================================
# MODEL 4: XGBoost (EXTREME GRADIENT BOOSTING)
# ==============================================================

print("="*70)
print("MODEL 4: XGBoost (EXTREME GRADIENT BOOSTING)")
print("="*70)

# ==============================================================
# BƯỚC 1: BASELINE (Default Parameters)
# ==============================================================

print("\nBƯỚC 1: CHẠY BASELINE (Default Hyperparameters)")
print("-"*70)

from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

# Tạo baseline model với default parameters
xgb_baseline = XGBClassifier(
    random_state=42,
    eval_metric='logloss',
    use_label_encoder=False
)

# Tạo pipeline: Transform → Model
xgb_baseline_pipeline = make_pipeline(col_transformer, xgb_baseline)

print("Default parameters:")
print("  - n_estimators: 100 (số trees)")
print("  - learning_rate: 0.3 (shrinkage factor)")
print("  - max_depth: 6 (độ sâu mỗi tree)")
print("  - subsample: 1.0 (dùng 100% samples)")
print("  - colsample_bytree: 1.0 (dùng 100% features)")

print("\nĐang training baseline...")
xgb_baseline_pipeline.fit(X_train_over, y_train_over)

# Dự đoán trên test set
y_pred_baseline_xgb = xgb_baseline_pipeline.predict(X_test)
y_pred_proba_baseline_xgb = xgb_baseline_pipeline.predict_proba(X_test)[:, 1]

# Tính metrics
baseline_accuracy_xgb = accuracy_score(y_test, y_pred_baseline_xgb)
baseline_f1_xgb = f1_score(y_test, y_pred_baseline_xgb)
baseline_auc_xgb = roc_auc_score(y_test, y_pred_proba_baseline_xgb)

print(f"\nBASELINE RESULTS:")
print(f"  Accuracy:  {baseline_accuracy_xgb:.4f} ({baseline_accuracy_xgb*100:.2f}%)")
print(f"  F1-Score:  {baseline_f1_xgb:.4f}")
print(f"  AUC Score: {baseline_auc_xgb:.4f}")

print(" Hoàn thành Baseline!")

### BƯỚC 2 - HYPERPARAMETER TUNING

In [ ]:
# ==============================================================
# BƯỚC 2: HYPERPARAMETER TUNING
# ==============================================================

print("\n" + "="*70)
print("BƯỚC 2: HYPERPARAMETER TUNING")
print("="*70)

# Định nghĩa hyperparameter grid
param_grid_xgb = {
    'xgbclassifier__n_estimators': [50, 100, 150],
    
    # Learning rate
    'xgbclassifier__learning_rate': [0.1, 0.15, 0.2],
    
    # Độ sâu tree
    'xgbclassifier__max_depth': [5, 7, 10],
    
    # Subsampling 
    'xgbclassifier__subsample': [0.8, 1.0],
    
    # Column sampling
    'xgbclassifier__colsample_bytree': [0.8, 1.0],
    
    # Regularization 
    'xgbclassifier__gamma': [0, 0.1, 0.5],
    
    # Handle imbalance
    'xgbclassifier__scale_pos_weight': [1, 1.38Ư]
}

print("\nHyperparameters để tuning:")
print("  1. n_estimators: [100, 200, 300]")
print("     → Số trees trong ensemble")
print("  2. learning_rate: [0.01, 0.05, 0.1, 0.3]")
print("     → Shrinkage factor (thấp → robust, cần nhiều trees)")
print("  3. max_depth: [3, 5, 7]")
print("     → Độ sâu mỗi tree (sâu → complex → overfit)")
print("  4.  subsample: [0.8, 1.0]")
print("     → Tỷ lệ samples mỗi tree (0.8 → stochastic → reduce overfit)")
print("  5. colsample_bytree: [0.8, 1.0]")
print("     → Tỷ lệ features mỗi tree (0.8 → diversity)")
print("  6. gamma: [0, 0.1, 0.5]")
print("     → Minimum loss reduction để split (lớn → simple)")
print("  7. scale_pos_weight: [1, 1.38]")
print("     → Weight cho minority class")

total_combinations = 3 * 4 * 3 * 2 * 2 * 3 * 2
print(f"\nTổng số combinations: {total_combinations}")
print(f"Với 5-fold CV: {total_combinations} x 5 = {total_combinations*5} fits")

# Tạo pipeline
xgb_pipeline = make_pipeline(
    col_transformer,
    XGBClassifier(random_state=42, eval_metric='logloss', use_label_encoder=False)
)

# GridSearchCV
grid_search_xgb = GridSearchCV(
    estimator=xgb_pipeline,
    param_grid=param_grid_xgb,
    cv=5,
    scoring='f1',
    n_jobs=-1,
    verbose=1
)

print("\nĐang chạy GridSearchCV...")
print("(Có thể mất 15-30 phút - XGBoost tốn thời gian nhất! )\n")

grid_search_xgb.fit(X_train_over, y_train_over)

print("\n GridSearchCV hoàn thành!")

In [1]:
# In ra best parameters
print("\nBEST HYPERPARAMETERS:")
print("="*70)

for param, value in grid_search_xgb.best_params_.items():
    clean_param = param.replace('xgbclassifier__', '')
    print(f"  {clean_param}: {value}")

print(f"\nBest Cross-Validation F1-Score: {grid_search_xgb.best_score_:.4f}")

# Lưu best parameters
best_n_est_xgb = grid_search_xgb.best_params_['xgbclassifier__n_estimators']
best_lr_xgb = grid_search_xgb.best_params_['xgbclassifier__learning_rate']
best_depth_xgb = grid_search_xgb.best_params_['xgbclassifier__max_depth']
best_subsample_xgb = grid_search_xgb.best_params_['xgbclassifier__subsample']
best_colsample_xgb = grid_search_xgb.best_params_['xgbclassifier__colsample_bytree']
best_gamma_xgb = grid_search_xgb.best_params_['xgbclassifier__gamma']
best_scale_xgb = grid_search_xgb.best_params_['xgbclassifier__scale_pos_weight']

# Giải thích
print("\nGiải thích Best Parameters:")
print(f"  • n_estimators = {best_n_est_xgb}:")
if best_n_est_xgb == 100:
    print(f"      → Default đã đủ, không cần thêm trees")
elif best_n_est_xgb == 200:
    print(f"      → Sweet spot - cân bằng tốt")
else:
    print(f"      → Cần nhiều trees - patterns phức tạp")

print(f"  • learning_rate = {best_lr_xgb}:")
if best_lr_xgb <= 0.05:
    print(f"      → Slow learning - robust, cần nhiều trees")
elif best_lr_xgb == 0.1:
    print(f"      → Balanced learning rate")
else:
    print(f"      → Fast learning - default works")

print(f"  • max_depth = {best_depth_xgb}:")
if best_depth_xgb <= 3:
    print(f"      → Shallow trees - simple patterns")
elif best_depth_xgb <= 5:
    print(f"      → Moderate depth - balanced")
else:
    print(f"      → Deep trees - complex patterns")

print(f"  • subsample = {best_subsample_xgb}:")
print(f"      → {'Stochastic boosting' if best_subsample_xgb < 1.0 else 'Dùng hết samples'}")

print(f"  • colsample_bytree = {best_colsample_xgb}:")
print(f"      → {'Feature subsampling' if best_colsample_xgb < 1.0 else 'Dùng hết features'}")

print(f"  • gamma = {best_gamma_xgb}:")
print(f"      → {'Aggressive splitting' if best_gamma_xgb == 0 else 'Conservative splitting'}")

print(f"  • scale_pos_weight = {best_scale_xgb}:")
print(f"      → {'SMOTE đã đủ' if best_scale_xgb == 1 else 'Cần weight thêm cho minority class'}")

# Lấy best model
xgb_best_model = grid_search_xgb.best_estimator_
print("\n Best model đã được lưu!")


BEST HYPERPARAMETERS:


NameError: name 'grid_search_xgb' is not defined

### BƯỚC 3: ĐÁNH GIÁ CHI TIẾT MODEL

In [ ]:
# ==============================================================
# BƯỚC 3: ĐÁNH GIÁ CHI TIẾT MODEL
# ==============================================================

print("\n" + "="*70)
print("BƯỚC 3: ĐÁNH GIÁ CHI TIẾT MODEL")
print("="*70)

# Dự đoán trên test set với best model
y_pred_test_xgb = xgb_best_model.predict(X_test)
y_pred_proba_test_xgb = xgb_best_model.predict_proba(X_test)[:, 1]

# --------------------------------------------------------------
# 3.1.  ACCURACY
# --------------------------------------------------------------
print("\n3.1. ACCURACY")
print("-"*70)

accuracy_test_xgb = accuracy_score(y_test, y_pred_test_xgb)
print(f"Test Accuracy (Tuned Model): {accuracy_test_xgb:.4f} ({accuracy_test_xgb*100:.2f}%)")
print(f"  → Model dự đoán ĐÚNG {accuracy_test_xgb*100:.1f}% trên test set")

# So sánh ngắn gọn với Baseline
improvement_xgb = (accuracy_test_xgb - baseline_accuracy_xgb) * 100
print(f"\nSo sánh với Baseline:")
print(f"  Baseline: {baseline_accuracy_xgb:.4f} → Tuned: {accuracy_test_xgb:.4f} ({improvement_xgb:+.2f}%)")

if improvement_xgb > 0:
    print(f"  Tuning đã cải thiện XGBoost!")
elif improvement_xgb == 0:
    print(f"  → Không thay đổi (default đã optimal)")
else:
    print(f"  → Default đã rất tốt, tuning giảm nhẹ (có thể overfit CV)")

In [ ]:
# --------------------------------------------------------------
# 3.2.  CLASSIFICATION REPORT
# --------------------------------------------------------------
print("\n3.2.  CLASSIFICATION REPORT")
print("-"*70)

print(classification_report(y_test, y_pred_test_xgb,
                           target_names=['Not Rejected (0)', 'Rejected (1)']))

# Tính metrics
precision_xgb = precision_score(y_test, y_pred_test_xgb)
recall_xgb = recall_score(y_test, y_pred_test_xgb)
f1_xgb = f1_score(y_test, y_pred_test_xgb)

print(f"\nMetrics cho class 'Rejected' (Class 1):")
print(f"  Precision: {precision_xgb:.4f} → {precision_xgb*100:.1f}% dự đoán 'Rejected' chính xác")
print(f"  Recall:    {recall_xgb:.4f} → Phát hiện {recall_xgb*100:.1f}% đơn thực sự bị rejected")
print(f"  F1-Score:  {f1_xgb:.4f} → Cân bằng giữa Precision và Recall")

# Nhận xét
print("\nNhận xét:")
if f1_xgb >= 0.85:
    print(f"  • F1={f1_xgb:.2f}: EXCELLENT!  XGBoost balance tốt")
elif f1_xgb >= 0.75:
    print(f"  • F1={f1_xgb:.2f}: GOOD - Performance tốt")
else:
    print(f"  • F1={f1_xgb:.2f}: Có thể cải thiện")

if recall_xgb >= 0.85:
    print(f"  • Recall cao ({recall_xgb:.2f}): Phát hiện tốt các đơn rejected")
elif recall_xgb >= 0.75:
    print(f"  • Recall trung bình ({recall_xgb:.2f}): Chấp nhận được")
else:
    print(f"  • Recall thấp ({recall_xgb:.2f}): Bỏ sót nhiều")

if precision_xgb >= 0.85:
    print(f"  • Precision cao ({precision_xgb:.2f}): Ít false alarms")
elif precision_xgb >= 0.75:
    print(f"  • Precision trung bình ({precision_xgb:.2f})")
else:
    print(f"  • Precision thấp ({precision_xgb:.2f}): Nhiều false alarms")

In [ ]:
# --------------------------------------------------------------
# 3.3.  CONFUSION MATRIX
# --------------------------------------------------------------
print("\n3.3.  CONFUSION MATRIX")
print("-"*70)

cm_xgb = confusion_matrix(y_test, y_pred_test_xgb)

# Vẽ confusion matrix
fig, ax = plt.subplots(figsize=(8, 6))
disp = ConfusionMatrixDisplay(confusion_matrix=cm_xgb,
                               display_labels=['Not Rejected', 'Rejected'])
disp.plot(cmap='Oranges', ax=ax, values_format='d')
plt.title(f'XGBoost - Confusion Matrix\n(n_estimators={best_n_est_xgb}, learning_rate={best_lr_xgb}, max_depth={best_depth_xgb})',
          fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Phân tích
tn_xgb, fp_xgb, fn_xgb, tp_xgb = cm_xgb.ravel()
print(f"\nChi tiết Confusion Matrix:")
print(f"  True Negatives (TN):  {tn_xgb:4d} → Dự đoán đúng 'Not Rejected'")
print(f"  False Positives (FP): {fp_xgb:4d} → Nhầm: dự đoán 'Rejected' nhưng thực tế 'Not Rejected'")
print(f"  False Negatives (FN): {fn_xgb:4d} → Nhầm: dự đoán 'Not Rejected' nhưng thực tế 'Rejected'")
print(f"  True Positives (TP):  {tp_xgb:4d} → Dự đoán đúng 'Rejected'")

total_errors = fp_xgb + fn_xgb
print(f"\nTổng lỗi: {total_errors} / {len(y_test)} = {total_errors/len(y_test)*100:.1f}%")

# Phân tích pattern lỗi
if fn_xgb > fp_xgb:
    print("  → False Negatives > False Positives")
    print("  → Model thiên về dự đoán 'Not Rejected' (conservative)")
else:
    print("  → False Positives > False Negatives")
    print("  → Model thiên về dự đoán 'Rejected' (aggressive)")

In [ ]:
# --------------------------------------------------------------
# 3.4.  ROC CURVE & AUC SCORE
# --------------------------------------------------------------
print("\n3.4. ROC CURVE & AUC SCORE")
print("-"*70)

# Tính ROC curve
fpr_xgb, tpr_xgb, thresholds_xgb = roc_curve(y_test, y_pred_proba_test_xgb)
auc_score_xgb = roc_auc_score(y_test, y_pred_proba_test_xgb)

# Vẽ ROC curve
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(fpr_xgb, tpr_xgb, color='darkorange', lw=2,
        label=f'XGBoost (AUC = {auc_score_xgb:.4f})')
ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--',
        label='Random Classifier (AUC = 0.5)')
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('False Positive Rate (FPR)', fontsize=12)
ax.set_ylabel('True Positive Rate (TPR)', fontsize=12)
ax.set_title(f'XGBoost - ROC Curve\n(n_estimators={best_n_est_xgb}, learning_rate={best_lr_xgb})',
             fontsize=14, fontweight='bold')
ax.legend(loc="lower right", fontsize=11)
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nAUC Score: {auc_score_xgb:.4f}")

# Đánh giá AUC
if auc_score_xgb >= 0.9:
    print(f"  → OUTSTANDING!  Model có discriminative power rất tốt")
elif auc_score_xgb >= 0.8:
    print(f"  → EXCELLENT! Model phân biệt 2 classes tốt")
elif auc_score_xgb >= 0.7:
    print(f"  → ACCEPTABLE, nhưng có thể cải thiện")
else:
    print(f"  → POOR, cần xem xét lại model")

In [ ]:
# --------------------------------------------------------------
# 3.5.  FEATURE IMPORTANCE ⭐
# --------------------------------------------------------------
print("\n3.5. FEATURE IMPORTANCE")
print("-"*70)

# Extract XGBoost model từ pipeline
xgb_model = xgb_best_model.named_steps['xgbclassifier']

# Get feature names sau transform
feature_names = xgb_best_model.named_steps['columntransformer'].get_feature_names_out()

# Get feature importance (default = 'gain')
importance_gain = xgb_model.feature_importances_

# Tạo DataFrame
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importance_gain
}).sort_values('Importance', ascending=False)

# Normalize to percentage
importance_df['Importance_Pct'] = (importance_df['Importance'] / importance_df['Importance'].sum()) * 100

print("\nTop 15 Features quan trọng nhất:\n")
print(importance_df.head(15).to_string(index=False))

# Plot Feature Importance
fig, ax = plt.subplots(figsize=(10, 8))
top_15 = importance_df.head(15)
bars = ax.barh(range(len(top_15)), top_15['Importance'], color='darkorange')

# Highlight top 5
for i in range(min(5, len(bars))):
    bars[i].set_color('darkred')

ax.set_yticks(range(len(top_15)))
ax.set_yticklabels(top_15['Feature'], fontsize=10)
ax.set_xlabel('Importance (Gain)', fontsize=12)
ax.set_title('XGBoost - Top 15 Feature Importance\n(Top 5 in Dark Red)',
             fontsize=14, fontweight='bold')
ax.invert_yaxis()

# Thêm percentage labels
for i, (idx, row) in enumerate(top_15.iterrows()):
    ax.text(row['Importance'], i, f"  {row['Importance_Pct']:.1f}%",
            va='center', fontsize=9)

plt.tight_layout()
plt.show()

print("\n💡 Giải thích:")
print("  • Importance = Gain (improvement in loss khi split)")
print("  • Features càng cao càng quan trọng cho predictions")
print("  • Business nên tập trung vào top features để cải thiện")

In [ ]:
# --------------------------------------------------------------
# 3.6. LƯU KẾT QUẢ VÀO BẢNG TỔNG HỢP
# --------------------------------------------------------------
print("\n3.6. LƯU KẾT QUẢ VÀO BẢNG TỔNG HỢP")
print("-"*70)

# Thêm kết quả XGBoost vào dataframe
results_df = pd.concat([results_df, pd.DataFrame([{
    'Model': 'XGBoost',
    'Baseline_Accuracy': baseline_accuracy_xgb,
    'Tuned_Accuracy': accuracy_test_xgb,
    'Precision': precision_xgb,
    'Recall': recall_xgb,
    'F1_Score': f1_xgb,
    'AUC_Score': auc_score_xgb
}])], ignore_index=True)

print(" Kết quả đã được lưu!")
print("\nBảng tổng hợp hiện tại:")
print(results_df.to_string(index=False))

# Tóm tắt
print("\n" + "="*70)
print("XGBoost - HOÀN THÀNH!")
print("="*70)
print(f"\nKết quả cuối cùng:")
print(f"  • Best n_estimators: {best_n_est_xgb}")
print(f"  • Best learning_rate: {best_lr_xgb}")
print(f"  • Best max_depth: {best_depth_xgb}")
print(f"  • Final Accuracy: {accuracy_test_xgb:.4f} ({accuracy_test_xgb*100:.2f}%)")
print(f"  • F1-Score: {f1_xgb:.4f}")
print(f"  • AUC Score: {auc_score_xgb:.4f}")
print(f"  • Training time: ~15-30 phút (864 combinations! )")

print("\nModel XGBoost đã sẵn sàng!")

# ============================================================
# LƯU MODEL & KẾT QUẢ
# ============================================================
import joblib

# Lưu model
joblib.dump(xgb_best_model, 'xgboost_tuned_pipeline.pkl')
print(" Đã lưu XGBoost model vào 'xgboost_tuned_pipeline.pkl'")

# Lưu kết quả CSV
results_df.to_csv('results_xgboost.csv', index=False)
print(" Đã lưu kết quả ra 'results_xgboost.csv'")